# Objetivo

Entrenar y testear la optimizacion de los portaforlios

Propiedades de los portafolios

$Retorno(P)=\sum{w_i}{Retorno(A_i)}$

Lo que quiere decir que el retorno total de un portafolio es igual al ponderado de sus retornos totales parciales.

Como Retorno(X) es la suma de todos los retornos individuales, entonces se trata de una variable conmutativa

In [19]:
import yfinance as yfin
import numpy as np
import yaml
from scipy.optimize import basinhopping,minimize
import logging

logging.basicConfig(level='INFO')

In [3]:
with open("config.yaml","r") as f:
    config = yaml.safe_load(f)

In [4]:
inicio='2015-01-01'
simbolos=config['Simbolos']

In [16]:
precios=yfin.download(simbolos,inicio)['Close']
precios=np.log(precios/precios.shift(1))
precios=precios.resample('W').sum()
precios

C:\Users\palaj\AppData\Local\Temp\ipykernel_15768\42433584.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  precios=yfin.download(simbolos,inicio)['Close']
[*********************100%***********************]  11 of 11 completed


Ticker,AAPL,ANET,CORT,CPRX,NVDA,PGR,SEZL,TPL,UNH,USLM,V
Date,,,,,,,,,,,
2015-01-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-01-11,0.024217,0.057956,-0.019418,0.013514,-0.009484,-0.004448,0.000000,0.043530,0.028755,0.019950,-0.017087
2015-01-18,-0.055243,-0.028713,-0.009852,-0.013514,0.001002,-0.020643,0.000000,-0.122867,0.019572,-0.015098,-0.021651
2015-01-25,0.063866,-0.017664,-0.047306,0.040005,0.036887,0.004541,0.000000,0.011040,0.057232,0.015098,0.013016
2015-02-01,0.036330,-0.004584,0.006897,0.176312,-0.075707,-0.020596,0.000000,0.047315,-0.052704,-0.059782,-0.013173
...,...,...,...,...,...,...,...,...,...,...,...
2025-11-23,-0.003383,-0.112175,0.022902,0.004305,-0.061203,0.007653,0.043167,-0.131732,-0.005889,-0.003512,-0.006201
2025-11-30,0.026749,0.106909,0.026934,0.005569,-0.010565,0.008251,0.125995,-0.036512,0.030168,0.021703,0.019505
2025-12-07,-0.000251,-0.016123,0.081127,0.005113,0.030163,-0.026126,0.102133,0.069825,0.003451,-0.000988,-0.009614


In [ ]:
class Optimizer():

    def __init__(self) -> None:
        self.val_score=1e6

    def optimizar(self,x0,args,valid_ds,constraints,bounds):
        result=minimize(self.loss_funct,x0,args=args,callback=(lambda x: self.early_stopping_callback(x,valid_ds)),constraints=constraints,bounds=bounds)
        return result.x,result.fun

    def loss_funct(self,w,Ra):
        R=np.dot(Ra,w)
        return self.suma_negativos(R)
    
    def suma_negativos(self,R):
        suma=0
        for r in R:
            if r<0:
                suma+=abs(r)
        return suma
    
    def early_stopping_callback(self,w,valid_ds):
        #logging.info(f"Evaluando loss function para w: {w}")
        
        val_score=self.loss_funct(w,valid_ds)
        #logging.info(f"Valid score generado : {val_score}")
        if val_score<self.val_score:
            self.val_score=val_score #Actualiza el mejor score de validacion actual
            #logging.info(f"Nuevo score: {self.val_score}")
            self.best_w=w
        else:
            logging.info(f"No hay mejoras para el valid_ds")
            return True

In [44]:
precios_train=precios[:'2020-01-01']
precios_valid=precios['2020-01-02':]

constraints = (
    {'type': 'ineq', 'fun': lambda w:config['PortOPT']['MaxLeverage']-np.sum(w)}, #Maximo el leverage especificado
    {'type': 'eq', 'fun': lambda w:config['PortOPT']['Target']*5-np.sum(np.dot(precios_train,w))} # Garantiza que se respete el target especificado
)

bounds=[(0, None) for _ in range(precios_train.shape[1])]

In [45]:
opt=Optimizer()
w,f=opt.optimizar(
    np.zeros(precios_train.shape[1]),
    args=(precios_train),
    valid_ds=precios_valid,
    constraints=constraints,
    bounds=bounds
)

INFO:root:Evaluando loss function para w: [1.55431223e-15 5.55111512e-17 1.30173650e-14 3.71820630e-14
 6.59196932e-01 2.51838738e-01 0.00000000e+00 3.13491181e-02
 7.29373171e-04 7.41767758e-15 4.67928534e-01]
INFO:root:Valid score generado : 5.201415795299326
INFO:root:Nuevo score: 5.201415795299326
INFO:root:Evaluando loss function para w: [6.46644860e-16 0.00000000e+00 2.19933026e-15 0.00000000e+00
 2.98543453e-01 6.08108998e-01 0.00000000e+00 1.38777878e-17
 6.70465202e-02 0.00000000e+00 9.17511573e-01]
INFO:root:Valid score generado : 5.28689697464418
INFO:root:No hay mejoras para el valid_ds
INFO:root:Evaluando loss function para w: [1.05286109e-16 1.73074275e-16 3.12710572e-16 0.00000000e+00
 2.39385045e-01 3.74698484e-01 0.00000000e+00 1.53564043e-01
 2.32702078e-01 7.48071402e-17 8.44545821e-01]
INFO:root:Valid score generado : 5.107463103983521
INFO:root:Nuevo score: 5.107463103983521
INFO:root:Evaluando loss function para w: [0.00000000e+00 2.21727238e-16 0.00000000e+00 0.0

In [46]:
opt.best_w

array([3.38110170e-17, 1.09632391e-15, 0.00000000e+00, 3.45519444e-16,
       3.18511943e-01, 4.87818235e-01, 0.00000000e+00, 2.78475553e-01,
       3.57546019e-01, 8.53674941e-17, 1.89998238e-01])

In [49]:
opt.loss_funct(opt.best_w,precios_valid)

4.760800022329505